In [2]:
#Importo lo necesario para spark
import findspark
findspark.init()
from pyspark.sql import SparkSession
import pyspark
from pyspark.sql.types import StructType, StructField, StringType, IntegerType
from pyspark.sql.functions import length, col,udf,row_number,expr,when

# inicio la sesion de spark
spark = SparkSession.builder \
    .appName("TuAppName") \
    .config("spark.executor.memory", "5g") \
    .config("spark.driver.memory", "25g") \
    .config("spark.network.timeout", "1000s") \
    .master("local[*]") \
    .getOrCreate()

    

In [4]:
# Cargo el dataset
ruta = './datasets/Yelp/review.json'
df_spark= spark.read.json(ruta)


In [5]:
df_spark.show()

+--------------------+----+-------------------+-----+--------------------+-----+--------------------+------+--------------------+
|         business_id|cool|               date|funny|           review_id|stars|                text|useful|             user_id|
+--------------------+----+-------------------+-----+--------------------+-----+--------------------+------+--------------------+
|XQfwVwDr-v0ZS3_Cb...|   0|2018-07-07 22:09:11|    0|KU_O5udG6zpxOg-Vc...|  3.0|If you decide to ...|     0|mh_-eMZ6K5RLWhZyI...|
|7ATYjTIgM3jUlt4UM...|   1|2012-01-03 15:28:18|    0|BiTunyQ73aT9WBnpR...|  5.0|I've taken a lot ...|     1|OyoGAe7OKpv6SyGZT...|
|YjUWPpI6HXG530lwP...|   0|2014-02-05 20:30:30|    0|saUsX_uimxRlCVr67...|  3.0|Family diner. Had...|     0|8g_iMtfSiwikVnbP2...|
|kxX2SOes4o-D3ZQBk...|   1|2015-01-04 00:01:03|    0|AqPFMleE6RsU23_au...|  5.0|Wow!  Yummy, diff...|     1|_7bHUi9Uuf5__HHc_...|
|e4Vwtrqf-wpJfwesg...|   1|2017-01-14 20:54:15|    0|Sx8TMOWLNuJBWer-0...|  4.0|Cute inter

In [6]:
#Veo las review de la columna text
for i in range(0, 100, 100):
     df_spark.select(col('text').substr(i + 1, 1000)).show(truncate=False,n=6)

+-----------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------+
|substring(text, 1, 1000)                                                                                                                                           

In [ ]:
# Puede verse que son largas, para disminuir el peso y tener mayor eficiencia voy a generar un analisis de sentimiento

In [10]:
# Analisis de sentimiento usando Bert
from transformers import BertTokenizer, BertForSequenceClassification
from torch.nn.functional import softmax
import torch

# Cargar el tokenizador y el modelo preentrenado de BERT
model_name = "bert-base-uncased"
tokenizer = BertTokenizer.from_pretrained(model_name)
model = BertForSequenceClassification.from_pretrained(model_name)

# Función para analizar sentimientos con BERT
def analyze_sentiment(text):
    if text is None:
      return 0
    # Tokenizar el texto y convertirlo en tensores
    inputs = tokenizer(text, return_tensors="pt", truncation=True, padding=True)
    
    # Obtener la salida del modelo
    outputs = model(**inputs)
    
    # Aplicar softmax para obtener las probabilidades de las clases
    probabilities = softmax(outputs.logits, dim=1)
    
    # Obtener la probabilidad de la clase positiva (puede ajustarse según tu tarea específica)
    positive_probability = probabilities[:, 1].item()
    
    return positive_probability



Some weights of BertForSequenceClassification were not initialized from the model checkpoint at bert-base-uncased and are newly initialized: ['classifier.bias', 'classifier.weight']
You should probably TRAIN this model on a down-stream task to be able to use it for predictions and inference.


In [11]:
# Aplico la funcion al dataframe
from pyspark.sql.types import FloatType
analyze_sentiment_udf = udf(analyze_sentiment, FloatType())
df_result = df_spark.withColumn("sentiment_score", analyze_sentiment_udf(col("text")))

In [12]:
df_result.show()

+--------------------+----+-------------------+-----+--------------------+-----+--------------------+------+--------------------+---------------+
|         business_id|cool|               date|funny|           review_id|stars|                text|useful|             user_id|sentiment_score|
+--------------------+----+-------------------+-----+--------------------+-----+--------------------+------+--------------------+---------------+
|XQfwVwDr-v0ZS3_Cb...|   0|2018-07-07 22:09:11|    0|KU_O5udG6zpxOg-Vc...|  3.0|If you decide to ...|     0|mh_-eMZ6K5RLWhZyI...|      0.5898219|
|7ATYjTIgM3jUlt4UM...|   1|2012-01-03 15:28:18|    0|BiTunyQ73aT9WBnpR...|  5.0|I've taken a lot ...|     1|OyoGAe7OKpv6SyGZT...|     0.55899507|
|YjUWPpI6HXG530lwP...|   0|2014-02-05 20:30:30|    0|saUsX_uimxRlCVr67...|  3.0|Family diner. Had...|     0|8g_iMtfSiwikVnbP2...|      0.5481514|
|kxX2SOes4o-D3ZQBk...|   1|2015-01-04 00:01:03|    0|AqPFMleE6RsU23_au...|  5.0|Wow!  Yummy, diff...|     1|_7bHUi9Uuf5__HHc

In [13]:
#Genero un criterio para poder definir un creterio 1:sentimiento negativo, 2:sentimiento positivo, 0:neutro
df_result = df_result.withColumn("sentiment",
                  when((col("stars") < 3) & (col("sentiment_score") > 0.5), 1)
                  .when((col("stars") < 3) & (col("sentiment_score") < 0.3), 1)
                  .when((col("stars") > 3) & (col("sentiment_score") < 0.3), 2)
                  .when((col("stars") > 3) & (col("sentiment_score") > 0.3), 2)
                  .otherwise(0))

In [14]:
df_result.show()

+--------------------+----+-------------------+-----+--------------------+-----+--------------------+------+--------------------+---------------+---------+
|         business_id|cool|               date|funny|           review_id|stars|                text|useful|             user_id|sentiment_score|sentiment|
+--------------------+----+-------------------+-----+--------------------+-----+--------------------+------+--------------------+---------------+---------+
|XQfwVwDr-v0ZS3_Cb...|   0|2018-07-07 22:09:11|    0|KU_O5udG6zpxOg-Vc...|  3.0|If you decide to ...|     0|mh_-eMZ6K5RLWhZyI...|      0.5898219|        0|
|7ATYjTIgM3jUlt4UM...|   1|2012-01-03 15:28:18|    0|BiTunyQ73aT9WBnpR...|  5.0|I've taken a lot ...|     1|OyoGAe7OKpv6SyGZT...|     0.55899507|        2|
|YjUWPpI6HXG530lwP...|   0|2014-02-05 20:30:30|    0|saUsX_uimxRlCVr67...|  3.0|Family diner. Had...|     0|8g_iMtfSiwikVnbP2...|      0.5481514|        0|
|kxX2SOes4o-D3ZQBk...|   1|2015-01-04 00:01:03|    0|AqPFMleE6Rs

In [15]:
#Elimino las columnas text y score queu ya no se usaran
df_result = df_result.drop("text")
df_result = df_result.drop("sentiment_score")


In [17]:
# Exporto el parquet
df_result.write.parquet("dataset_review/review.parquet")
spark.stop()

23/11/22 17:20:00 WARN NettyRpcEnv: Ignored failure: java.util.concurrent.TimeoutException: Cannot receive any reply from 192.168.0.20:52502 in 10000 milliseconds
23/11/22 17:19:59 WARN Executor: Issue communicating with driver in heartbeater
org.apache.spark.rpc.RpcTimeoutException: Futures timed out after [10000 milliseconds]. This timeout is controlled by spark.executor.heartbeatInterval
	at org.apache.spark.rpc.RpcTimeout.org$apache$spark$rpc$RpcTimeout$$createRpcTimeoutException(RpcTimeout.scala:47)
	at org.apache.spark.rpc.RpcTimeout$$anonfun$addMessageIfTimeout$1.applyOrElse(RpcTimeout.scala:62)
	at org.apache.spark.rpc.RpcTimeout$$anonfun$addMessageIfTimeout$1.applyOrElse(RpcTimeout.scala:58)
	at scala.runtime.AbstractPartialFunction.apply(AbstractPartialFunction.scala:38)
	at org.apache.spark.rpc.RpcTimeout.awaitResult(RpcTimeout.scala:76)
	at org.apache.spark.rpc.RpcEndpointRef.askSync(RpcEndpointRef.scala:101)
	at org.apache.spark.executor.Executor.reportHeartBeat(Executor.s

Py4JJavaError: An error occurred while calling o104.parquet.
: org.apache.spark.SparkException: Job aborted due to stage failure: Task 2 in stage 6.0 failed 1 times, most recent failure: Lost task 2.0 in stage 6.0 (TID 47) (192.168.0.20 executor driver): org.apache.spark.SparkException: Python worker exited unexpectedly (crashed)
	at org.apache.spark.api.python.BasePythonRunner$ReaderIterator$$anonfun$1.applyOrElse(PythonRunner.scala:612)
	at org.apache.spark.api.python.BasePythonRunner$ReaderIterator$$anonfun$1.applyOrElse(PythonRunner.scala:594)
	at scala.runtime.AbstractPartialFunction.apply(AbstractPartialFunction.scala:38)
	at org.apache.spark.sql.execution.python.BasePythonUDFRunner$$anon$1.read(PythonUDFRunner.scala:99)
	at org.apache.spark.sql.execution.python.BasePythonUDFRunner$$anon$1.read(PythonUDFRunner.scala:75)
	at org.apache.spark.api.python.BasePythonRunner$ReaderIterator.hasNext(PythonRunner.scala:525)
	at org.apache.spark.InterruptibleIterator.hasNext(InterruptibleIt

23/11/22 17:26:44 WARN PythonUDFRunner: Incomplete task 4.0 in stage 6 (TID 49) interrupted: Attempting to kill Python Worker
23/11/22 17:26:44 WARN TaskSetManager: Lost task 4.0 in stage 6.0 (TID 49) (192.168.0.20 executor driver): TaskKilled (Stage cancelled: Job aborted due to stage failure: Task 2 in stage 6.0 failed 1 times, most recent failure: Lost task 2.0 in stage 6.0 (TID 47) (192.168.0.20 executor driver): org.apache.spark.SparkException: Python worker exited unexpectedly (crashed)
	at org.apache.spark.api.python.BasePythonRunner$ReaderIterator$$anonfun$1.applyOrElse(PythonRunner.scala:612)
	at org.apache.spark.api.python.BasePythonRunner$ReaderIterator$$anonfun$1.applyOrElse(PythonRunner.scala:594)
	at scala.runtime.AbstractPartialFunction.apply(AbstractPartialFunction.scala:38)
	at org.apache.spark.sql.execution.python.BasePythonUDFRunner$$anon$1.read(PythonUDFRunner.scala:99)
	at org.apache.spark.sql.execution.python.BasePythonUDFRunner$$anon$1.read(PythonUDFRunner.scala: